In [12]:
!pip install ipythonblocks

#We import all the necessary libraries.
from ipythonblocks import BlockGrid
from agents import *

from search import ( # Bases for problem building
    Problem, Node, Graph, UndirectedGraph,
    SimpleProblemSolvingAgentProgram,
    GraphProblem
)

from search import ( # Uninformed search algorithms
    tree_search, graph_search, best_first_graph_search,
    breadth_first_tree_search, breadth_first_search,
    depth_first_tree_search, depth_first_graph_search,
    depth_limited_search, iterative_deepening_search,
    uniform_cost_search,
    compare_searchers
)

from search import ( # Informed search algorithms
    greedy_best_first_graph_search, astar_search
)

# Environment

In [13]:
class Ball(Thing):
    pass

class Click(Thing):
    pass

#We define the environment class
class CleanupPuzzleEnvironment(Environment):
    def __init__(self, width=5, height=5):
        #We define the main characteristics of the environment. Among them, the color of the empty cells,
        #the size of the grid, and the color of the balls.
        super(CleanupPuzzleEnvironment, self).__init__()
        self.width = width
        self.height = height

        self.bg_color = (207, 216, 220)
        self.ball_color = (0, 191, 165)
        self.click_color = (244, 67, 54)
        self.grid = BlockGrid(width, height, fill=self.bg_color)

    def __str__(self):
        #We display the initial grid grid.
        world = self.get_world()
        self.draw_grid(world)
        self.grid.show()
        return ''

    def draw_grid(self, world):
        #We build the displayable grid, by showing first the grid as empty.
        #We then proceed to check all the spaces of the grid, to fill it with either clicks, balls, or leaving
        #the space empty
        self.grid[:] = self.bg_color
        for x in range(0, len(world)):
            for y in range(0, len(world[x])):
                if len(world[x][y]) and isinstance(world[x][y][0], Ball):
                    self.grid[x, y] = self.ball_color
                elif len(world[x][y]) and isinstance(world[x][y][0], Click):
                    self.grid[x, y] = self.click_color

    def get_world(self):
        #Returns the items that are in the world at the moment
        result = []
        for x in range(self.height):
            row = []
            for y in range(self.width):
                row.append(self.list_things_at((x, y)))
            result.append(row)
        return result

    def is_inbounds(self, location):
        #Checks that the location selected is inside the boundaries of the problem.
        x,y = location
        return not (x < 0 or x >= self.height or y < 0 or y >= self.width)


# Problem

In [86]:
class CleanBoardProblem(Problem):
    #We define the base problem
    def __init__(self, initial=[], goal=[], size=5):
        Problem.__init__(self, initial, goal)
        self.size = size
        self.initial_length = len(initial)

    def is_inbounds(self, location):
        #Check that the actions do not affect out of bound parts of the grid.
        x,y = location
        return not (x < 0 or x >= self.size or y < 0 or y >= self.size)

    def actions(self, state):
        #We get a list of all the actions that could be performed by the agent, and what areas of the grid it affects.
        action_coords = []

        for (x, y) in state:
            near_locations = [(x-1, y), (x+1, y), (x,y-1), (x, y+1)]
            for loc in near_locations:
                if self.is_inbounds(loc) and loc not in state and loc not in action_coords:
                    action_coords.append(loc)

        sorted_list = sorted(action_coords, key=lambda x: [x[0], x[1]])
        return tuple(sorted_list)

    def result(self, state, action):
        #We get the resulting states that could be obtained from the actions performed by the action.
        x, y = action
        near_locations = [(x-1, y), (x+1, y), (x,y-1), (x, y+1)]

        new_state = [e for e in state]
        for loc in near_locations:
            if self.is_inbounds(loc) and loc not in new_state:
                new_state.append(loc)
            elif self.is_inbounds(loc) and loc in new_state:
                new_state.remove(loc)

        sorted_list = sorted(new_state, key=lambda x: [x[0], x[1]])
        return tuple(sorted_list)

    def goal_test(self, state):
        #Check the current state to determine if we have reached the goal.
        if len(state) < 1:
            return True
        return False

    #We get the cost of each of the actions and add them together.
    def path_cost(self, c, state1, action, state2):
        return c + len(state2)
        # return c + 1

# Visualization Function

In [73]:
def display_solution(solution, initial_state, size):
    #We print the non-solution result. The case were the agent cant reach a solution.
    if not solution:
        print("Failure: no solution found")
        return

    #We print the initial state of the problem.
    print('Initial State:')
    environment = CleanupPuzzleEnvironment(size, size)
    for loc in initial_state:
        ball = Ball()
        environment.add_thing(ball, loc)
    print(environment)

    #We print all the subsequent steps, until we reach the final state.
    i = 0
    path = solution.path()
    for p in path:
        print('Step %s:' % (i+1))
        environment = CleanupPuzzleEnvironment(size, size)

        sol = solution.solution()
        if i < len(sol):
            click = Click()
            environment.add_thing(click, sol[i])

        for loc in p.state:
            ball = Ball()
            environment.add_thing(ball, loc)
        
        print(environment)
        print('')
        i += 1

# Heuristics

In [93]:
def heuristic_one(node):
    return len(node.state)

def heuristic_two(node):
    state = node.state

    #We get a list of all the actions that could be performed by the agent, and what areas of the grid it affects.
    action_coords = []

    for (x, y) in state:
        near_locations = [(x-1, y), (x+1, y), (x,y-1), (x, y+1)]
        for loc in near_locations:
            if loc not in state and loc not in action_coords:
                action_coords.append(loc)

    sorted_list = sorted(action_coords, key=lambda x: [x[0], x[1]])

    max_balls = 0
    for (x, y) in action_coords:
        near_locations = [(x-1, y), (x+1, y), (x,y-1), (x, y+1)]
        num_balls = 0
        for loc in near_locations:
            if loc in state:
                num_balls += 1
        
        if num_balls == 4:
            max_balls = 4
            break
        
        if max_balls < num_balls:
            max_balls = num_balls

    return len(state) - max_balls


# Test

In [96]:
initial_state = tuple([(0,0), (1,1), (2,0), (3,3), (4,2), (4,4)])
initial_state = tuple([(1,0), (2,1), (2,4), (3,0), (3,3), (3,5), (4,2), (4,4), (5,1), (5,3)])
initial_state = tuple([(0,3), (1,2), (1,4), (2,3), (3,7), (4,6), (4,3), (5,2), (5,4), (6,3), (5,9), (6,8), (6,9), (7,8), (7,10), (8,0), (8,5), (8,9), (9,1), (9,4), (9,6), (10,0), (10,5)])
size = 11

environment = CleanupPuzzleEnvironment(size, size)

for s in initial_state:
    ball = Ball()
    environment.add_thing(ball, s)

print(environment)

,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,


In [104]:
# initial_state = [(0,0), (1,1), (2,0), (4,3), (3,2), (3,4), (1,2), (0,3), (1,4)]
# initial_state = [(1,0), (2,1), (2,4), (3,0), (3,3), (3,5), (4,2), (4,4), (5,1), (5,3)]
initial_state = [(0,3), (1,2), (1,4), (2,3), (3,7), (4,6), (4,3), (5,2), (5,4), (6,3), (5,9), (6,8), (6,9), (7,8), (7,10), (8,0), (8,5), (8,9), (9,1), (9,4), (9,6), (10,0), (10,5)]

initial_state = sorted(initial_state, key=lambda x: [x[0], x[1]])
initial_state = tuple(initial_state)

size = 11

problem = CleanBoardProblem(initial_state, [], size)
solution = astar_search(problem, heuristic_one)

display_solution(solution, initial_state, size)

solution.path_cost

Initial State:


,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,



Step 1:


,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,




Step 2:


...



Step 7:


,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,




Step 8:


,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,


60

In [103]:
# initial_state = [(0,0), (1,1), (2,0), (4,3), (3,2), (3,4), (1,2), (0,3), (1,4)]
# initial_state = [(1,0), (2,1), (2,4), (3,0), (3,3), (3,5), (4,2), (4,4), (5,1), (5,3)]
initial_state = [(0,3), (1,2), (1,4), (2,3), (3,7), (4,6), (4,3), (5,2), (5,4), (6,3), (5,9), (6,8), (6,9), (7,8), (7,10), (8,0), (8,5), (8,9), (9,1), (9,4), (9,6), (10,0), (10,5)]

initial_state = sorted(initial_state, key=lambda x: [x[0], x[1]])
initial_state = tuple(initial_state)

size = 11

problem = CleanBoardProblem(initial_state, [], size)
solution = greedy_best_first_graph_search(problem, heuristic_one)

display_solution(solution, initial_state, size)

solution.path_cost

Initial State:


,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,



Step 1:


,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,




Step 2:


...



Step 15:


,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,




Step 16:


,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,


86